Test code to add skipped exons to a GTF

In [100]:
import CGAT.GTF as GTF
import CGAT.IOTools as IOTools

import copy 
import numpy as np

import collections

In [5]:
infile = '../../run/annotations/sequins.gtf.gz'
outfile = 'test.gtf.gz'

In [93]:
fraction = 1

transcript2exonCounts = {}

gtfs = GTF.iterator(IOTools.openFile(infile))
for gtf_lines in GTF.transcript_iterator(gtfs, strict=0):
    if gtf_lines[0].feature == 'exon':
        transcript_id = gtf_lines[0].transcript_id
        if len(gtf_lines)>2:
            transcript2exonCounts[transcript_id] = len(gtf_lines)

number_transcripts = len(transcript2exonCounts)

skip_transcripts = list(np.random.choice(
        list(transcript2exonCounts.keys()),
        size=int(number_transcripts * fraction), replace=True))

with IOTools.openFile(outfile, 'w') as outf:
    gtfs = GTF.iterator(IOTools.openFile(infile))
    for gtf_lines in GTF.transcript_iterator(gtfs, strict=0):
        if gtf_lines[0].feature == 'exon':
            
            # write out every transcript_id already in gtf
            for gtf in gtf_lines:
                outf.write("%s\n" % str(gtf))
            
            # if transcript has been selected, remove an exon at random and output as a new transcript
            transcript_id = gtf_lines[0].transcript_id
            if transcript_id in skip_transcripts:
                for repeat in range(0, skip_transcripts.count(transcript_id)):
                    transcript_id = gtf_lines[0].transcript_id

                    # don't want to remove first or last exon
                    remove_exon = np.random.choice(
                        transcript2exonCounts[transcript_id] - 2, 1)

                    new_transcript_id = "%s_skipped_%i" % (transcript_id, repeat)
                    for ix, gtf in enumerate(gtf_lines):
                        if ix != remove_exon:
                            gtf2 = copy.copy(gtf)
                            gtf2.setAttribute('transcript_id', new_transcript_id)
                            outf.write("%s\n" % str(gtf2))

In [96]:
transcript2exons = {}
transcript2exons['a'] = set((1,2,3,4))
transcript2exons['b'] = set((1,2,3,4))

In [97]:
print(                    [t_id for t_id in transcript2exons.keys()
                     for exon in transcript2exons[t_id]])

['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b']
